In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import Counter
import socket 
import threading

In [ ]:
def connect():   
    soc = socket.socket()
    global conn,isConn
    soc.bind(('localhost',9000))
    soc.listen(5)
    conn , addr = soc.accept()
    isConn = 1
    print("device connected")

In [ ]:
def SendMess(x):
    msg = bytes(f"{x}",'utf-8')
    conn.send(msg)

In [ ]:


def capture_and_predict(num_frames=20):
    model = YOLO("Best_Model.pt")
    camera = cv2.VideoCapture(0)
    captured_frames = []

    for _ in range(num_frames):
        ret, frame = camera.read()

        if not ret:
            print("Failed to grab frame")
            break

        cv2.imshow("test", frame)
        captured_frames.append(frame)
        key = cv2.waitKey(1)
        if key == 27:
             break
    camera.release()
    cv2.destroyAllWindows()

    # Predict using the captured frames
    predicted_classes = []
    results = model(captured_frames)  # list of Results objects -> perform inference using the model
    names = model.names

    for r in results:
        for c in r.boxes.cls:
            predicted_classes.append(names[int(c)])

    # Find the most frequent predicted class
    most_common_class = Counter(predicted_classes).most_common(1)
    final_predicted_class = most_common_class[0][0]

    # Save the final_predicted_class to a text file
    with open("predicted_class.txt", "w") as file:
        file.write(final_predicted_class)

    print("Predicted class saved to 'predicted_class.txt'")



SendMess("Connected")

In [ ]:
thread1 = threading.Thread(target=connect)
thread1.start()
thread1.join()

thread2 = threading.Thread(target=capture_and_predict)
thread2.start()
thread2.join()